Атестация по модулю - ВВедение в машинное обучение
В Базе Данных (инструкция по доступу) есть таблица , в которой содержатся сведения о звездах, полученные в ходе исследовании вселенной High Time Resolution Universe Survey (описание признаков можно найти в блоке Attribute Information) с целью определения одного из типа нейтронных звезд — пульсаров. Поле  таблицы  является откликом, все остальные поля — предикторы.

Вам необходимо получить выборку из таблицы  с помощью запросов на основании следующих критериев:
SQL - делали в учебном модуле: SELECT * FROM pulsar_stars WHERE (TARGET = 0 AND MIP BETWEEN 104.1953125 AND 104.5859375) OR (TARGET = 1 AND MIP BETWEEN 101.9609375 AND 115.3515625)
Все строки таблицы, где  и ;TARGET = 0 AND MIP BETWEEN 104.1953125 AND 104.5859375
Все строки таблицы, где  и .TARGET = 1 AND MIP BETWEEN 101.9609375 AND 115.3515625
Укажите число строк в полученной выборке:
Определите выборочное среднее для столбца :МИР
Выполните линейную нормировку всех значений предикторов полученной выборки.
Определите выборочное среднее для столбца  МИР после нормировки:
Обучите модель логистической регрессии, используя полученную после нормировки выборку в качестве тренировочного набора данных. Используйте LogisticRegression(random_state=2019, solver='lbfgs'), остальные параметры по умолчанию.
Выполните классификацию новой звезды (с помощью лог.регрессии):
Введите вероятность отнесения звезды 0.157, 0.311, 0.676, 0.586, 0.307,0.848, 0.673, 0.64 к классу пульсар:
Выполните классификацию новой звезды, с помощью метода к-ближайших соседей, используя нормализованные данные выборки.
Введите расстояние от новой звезды до ближайшего соседа, используя евклидову метрику.

In [221]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
#подключаем модель  регрессии из библиотеки sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [222]:
#чтение данных
df = pd.read_csv("/Users/ekaterina/github_repos/ITMO_STUDY/ datasets/ATTESTATION_3_MIP.csv", delimiter=',')
df.head()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
0,104.507812,50.274403,0.268410,-0.169219,1.729933,17.057981,11.706642,147.034435,0
1,104.468750,49.467309,0.328236,-0.094803,2.026756,15.993392,10.004127,115.104641,0
2,105.046875,47.085504,0.408034,0.331754,3.540970,26.129805,7.578517,57.367713,1
3,110.195312,54.475490,0.489785,-0.031921,19.280100,56.002144,2.875491,6.861892,1
4,104.328125,41.707521,0.481347,1.748107,6.112876,31.839215,5.593786,31.256371,0


In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   MIP     203 non-null    float64
 1   STDIP   203 non-null    float64
 2   EKIP    203 non-null    float64
 3   SIP     203 non-null    float64
 4   MC      203 non-null    float64
 5   STDC    203 non-null    float64
 6   EKC     203 non-null    float64
 7   SC      203 non-null    float64
 8   TARGET  203 non-null    int64  
dtypes: float64(8), int64(1)
memory usage: 14.4 KB


In [224]:
df.tail()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
198,112.304688,47.301486,0.459260,0.140741,60.255017,88.667228,1.075925,-0.437491,1
199,104.570312,38.543945,0.534334,1.451721,1.512542,16.051985,12.107143,156.803853,0
200,104.398438,54.915324,0.448083,-0.339692,1.123746,15.160780,14.893325,227.302601,0
201,104.218750,48.167341,0.107524,0.007509,2.982441,18.779495,7.932378,71.134676,0
202,105.414062,47.067526,0.877089,0.891877,67.658027,89.168460,0.708237,-1.273890,1


In [225]:
df.describe()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
count,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,106.454703,46.468291,0.464643,0.620862,14.401865,32.130714,7.532957,89.952629,0.497537
std,3.431455,5.452357,0.248717,0.847627,27.021907,23.776106,4.614874,96.262636,0.501230
min,101.960938,34.787229,-0.018665,-0.705473,0.473244,9.305757,-0.696153,-1.801220,0.000000
25%,104.316406,42.760889,0.316218,0.030672,1.934783,15.459401,3.519391,11.308542,0.000000
50%,104.531250,46.467709,0.420093,0.446316,3.210702,20.819362,7.749978,64.401610,0.000000
75%,108.207031,50.018212,0.561121,1.033490,14.120401,47.888167,10.555280,136.830241,1.000000
max,115.351562,61.287651,1.396708,4.715823,149.843646,101.361763,22.810861,559.679864,1.000000


In [226]:
df_1 = df.drop(['TARGET'], axis=1)
df_1


,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC
0,104.507812,50.274403,0.268410,-0.169219,1.729933,17.057981,11.706642,147.034435
1,104.468750,49.467309,0.328236,-0.094803,2.026756,15.993392,10.004127,115.104641
2,105.046875,47.085504,0.408034,0.331754,3.540970,26.129805,7.578517,57.367713
3,110.195312,54.475490,0.489785,-0.031921,19.280100,56.002144,2.875491,6.861892
4,104.328125,41.707521,0.481347,1.748107,6.112876,31.839215,5.593786,31.256371
...,...,...,...,...,...,...,...,...
198,112.304688,47.301486,0.459260,0.140741,60.255017,88.667228,1.075925,-0.437491
199,104.570312,38.543945,0.534334,1.451721,1.512542,16.051985,12.107143,156.803853
200,104.398438,54.915324,0.448083,-0.339692,1.123746,15.160780,14.893325,227.302601
201,104.218750,48.167341,0.107524,0.007509,2.982441,18.779495,7.932378,71.134676


In [227]:
'''
рабочий код, но нужен мин макс скалер
d = preprocessing.normalize(df_1)

names = list(df_1.columns.values.tolist())
scaled_df = pd.DataFrame(d, columns=names)
scaled_df.head()
'''

'\nd = preprocessing.normalize(df_1)\n\nnames = list(df_1.columns.values.tolist())\nscaled_df = pd.DataFrame(d, columns=names)\nscaled_df.head()\n'

In [228]:
df['TARGET'].value_counts()

0    102
1    101
Name: TARGET, dtype: int64

In [229]:
from sklearn.preprocessing import MinMaxScaler
MinMaxScalar = preprocessing.MinMaxScaler()
col = df_1.columns
result = MinMaxScalar.fit_transform(df_1)
scaled_df = pd.DataFrame(result,columns = col)
scaled_df


,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC
0,0.190198,0.584412,0.202827,0.098916,0.008413,0.084212,0.527621,0.265077
1,0.187281,0.553956,0.245095,0.112643,0.010400,0.072647,0.455195,0.208210
2,0.230455,0.464078,0.301475,0.191325,0.020538,0.182759,0.352009,0.105380
3,0.614936,0.742941,0.359234,0.124242,0.125908,0.507261,0.151940,0.015429
4,0.176779,0.261139,0.353272,0.452582,0.037756,0.244780,0.267577,0.058876
...,...,...,...,...,...,...,...,...
198,0.772462,0.472229,0.337667,0.156091,0.400225,0.862100,0.075385,0.002429
199,0.194866,0.141761,0.390709,0.397911,0.006958,0.073284,0.544659,0.282476
200,0.182030,0.759539,0.329770,0.067471,0.004355,0.063603,0.663184,0.408035
201,0.168611,0.504902,0.089156,0.131515,0.016798,0.102913,0.367062,0.129899


In [230]:
scaled_df.describe()

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC
count,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,0.335590,0.440788,0.341471,0.244653,0.093249,0.247946,0.350070,0.163414
std,0.256258,0.205746,0.175726,0.156351,0.180905,0.258279,0.196319,0.171444
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.175904,0.300888,0.236604,0.135788,0.009785,0.066847,0.179331,0.023349
50%,0.191949,0.440766,0.309995,0.212456,0.018327,0.125072,0.359303,0.117907
75%,0.466453,0.574745,0.409635,0.320765,0.091365,0.419119,0.478642,0.246903
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [231]:
X = scaled_df
y = df['TARGET']

In [232]:

lr = LogisticRegression(solver='lbfgs', random_state=2019)

In [233]:
clf=lr.fit(X, y)

In [234]:
test = np.array([0.157, 0.311, 0.676, 0.586, 0.307,0.848, 0.673, 0.64])

In [235]:
test.reshape(1,-1)

array([[0.157, 0.311, 0.676, 0.586, 0.307, 0.848, 0.673, 0.64 ]])

In [236]:
clf.predict(test.reshape(1,-1)) #0.676, 0.586, 0.307,0.848, 0.673, 0.64  #([df.loc[719, features].tolist()])[0]

array([1])

In [237]:
clf.predict_proba(test.reshape(1,-1))

array([[0.40752863, 0.59247137]])

In [238]:
#Выполните классификацию новой звезды, с помощью метода -ближайших соседей, используя нормализованные данные выборки.
#Введите расстояние от новой звезды до ближайшего соседа, используя евклидову метрику.

neigh = KNeighborsClassifier(n_neighbors=1, p=2)
neigh.fit(X, y)


KNeighborsClassifier(n_neighbors=1)

In [239]:
#  Задаем наш новый объект с координатами
NewObject = test

In [240]:
# Назначенный класс:
neigh.predict([NewObject])

array([1])

In [241]:
# Вывод расстояний до  𝑘  соседей в порядке удаления (первый массив — расстояния, второй — идентификаторы объектов). 
# ции в таблице.
neigh.kneighbors([NewObject])

(array([[0.91092225]]), array([[73]]))